Testing the Prism (https://prism.oregonstate.edu/downloads/) precipitation data to see if this should be one of the variables in my model. The precipitation data is supposed to be highly accurate as it uses ground readings. However, the data also go through updates as more data is recorded. I will be using the first iteration of the data as I need to work with real time data, so I'm not sure if it will be superior (or robust enough) in comparison with something like GFS, which is the most real time, but only uses land models and is low spatial resolution. Another alternative to PRISM is GridMET data through Microsft Planetary computer (https://planetarycomputer.microsoft.com/dataset/gridmet)

The .xml file received with the HTTP download describes the data like this: "Display and/or analysis requiring spatially distributed daily precipitation for June 23, 2025. There are many methods of interpolating climate from monitoring stations to grid points. Some provide estimates of acceptable accuracy in flat terrain, but few have been able to adequately explain the extreme, complex variations in climate that occur in mountainous regions. Significant progress in this area has been achieved through the development of PRISM (Parameter-elevation Regressions on Independent Slopes Model). PRISM is an analytical model that uses point data and an underlying grid such as a digital elevation model (DEM) or a 30 yr climatological average to generate gridded estimates of daily precipitation and temperature (as well as other climatic parameters). PRISM is well suited to regions with mountainous terrain, because it incorporates a conceptual framework that addresses the spatial scale and pattern of orographic processes. Grids were modeled on a daily basis"

http url template: https://services.nacse.org/prism/data/get/<region>/<res>/<element>/<date><?format=[nc|asc|bil]>

In [ ]:
import requests
import zipfile
import io
import xarray as xr

In [ ]:
date = 20250623

url = f"https://services.nacse.org/prism/data/get/us/800m/ppt/{date}?format=nc"

response = requests.get(url)

    


In [ ]:
if response.ok:
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        # Print list of files in the zip
        print("Contents of zip:", z.namelist())
        
        nc_filename = [name for name in z.namelist() if name.endswith(".nc")][0]

        z.extract(nc_filename, path="data/prism")
        # ds = xr.open_dataset(f"./downloads/{nc_filename}")

        # with open(f"data/prism/prism_ppt_{date}.zip", "wb") as f:
        #     f.write(response.content)
        #     print("download successful")
else:
    print(f"Error {response.status_code}: {response.text}")

In [ ]:
ds = xr.open_dataset("data/prism/prism_ppt_20250623/prism_ppt_us_30s_20250623.nc", engine="netcdf4")


In [ ]:
ds

In [ ]:
ds['Band1'].plot(robust=True)

In [ ]:
ds['Band1'].mean()

In [ ]:
import rioxarray as rxr

ds_prism = ds.rio.write_crs("EPSG:4269")
ds_prism_5070 = ds_prism.rio.reproject("EPSG:5070")

In [ ]:
ds_prism_5070.plot(robust=True)

In [ ]:
ds_prism_5070.to_netcdf("data/prism/prism_5070.nc")

Prism precipiation data is in mm